# OKCupid Date a Scientist - Part 2 Match couples
Given clients' dataset of OKcupid - a dating company, this project's objective is to cluster these clients into groups and try to match a couple. There is only one data file which is 'profile.csv'. The dataset consists of 31 columns which are about clients. All columns are as follows:
 - **age**
 - **body_type** nominal such as average, thin, a little extra, etc.
 - diet 
 - drinks 
 - drugs        
 - education       
 - **essay0 - essay9**  These ten columns are more details about themselves which written by each individual, for examples, 'About me', 'What I do' and 'My interests'. However, I have not been provided the real questions or topics for each essay column.
 - ethnicity such as asian, indian, white, etc.
 - height           
 - income      
 - job           
 - last_online       
 - location         
 - offspring      
 - orientation      
 - pets         
 - religion      
 - sex             
 - sign - zodiac signs such as aries, taurus, gemini    
 - smokes         
 - speaks 
 - status - a member's status such as single, available, married

In [1]:
# import necessary libraries and change some display settings.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [57]:
# import dataset from a csv file
df = pd.read_csv("profiles.csv")

In [58]:
# check all columns' names
df.columns

Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'essay0',
       'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7',
       'essay8', 'essay9', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'orientation', 'pets',
       'religion', 'sex', 'sign', 'smokes', 'speaks', 'status'],
      dtype='object')

In [59]:
# have a look at the first five rows of data
df['religion_cleaned'] = df.religion.str.split().str.get(0)
df.head(10)

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status,religion_cleaned
0,22,a little extra,strictly anything,socially,never,working on college/university,"about me:<br />\n<br />\ni would love to think that i was some some kind of intellectual:\neither the dumbest smart guy, or the smartest dumb guy. can't say i\ncan tell the difference. i love to talk about ideas and concepts. i\nforge odd metaphors instead of reciting cliches. like the\nsimularities between a friend of mine's house and an underwater\nsalt mine. my favorite word is salt by the way (weird choice i\nknow). to me most things in life are better as metaphors. i seek to\nmake myself a little better everyday, in some productively lazy\nway. got tired of tying my shoes. considered hiring a five year\nold, but would probably have to tie both of our shoes... decided to\nonly wear leather shoes dress shoes.<br />\n<br />\nabout you:<br />\n<br />\nyou love to have really serious, really deep conversations about\nreally silly stuff. you have to be willing to snap me out of a\nlight hearted rant with a kiss. you don't have to be funny, but you\nhave to be able to make me laugh. you should be able to bend spoons\nwith your mind, and telepathically make me smile while i am still\nat work. you should love life, and be cool with just letting the\nwind blow. extra points for reading all this and guessing my\nfavorite video game (no hints given yet). and lastly you have a\ngood attention span.","currently working as an international agent for a freight\nforwarding company. import, export, domestic you know the\nworks.<br />\nonline classes and trying to better myself in my free time. perhaps\na hours worth of a good book or a video game on a lazy sunday.","making people laugh.<br />\nranting about a good salting.<br />\nfinding simplicity in complexity, and complexity in simplicity.","the way i look. i am a six foot half asian, half caucasian mutt. it\nmakes it tough not to notice me, and for me to blend in.","books:<br />\nabsurdistan, the republic, of mice and men (only book that made me\nwant to cry), catcher in the rye, the prince.<br />\n<br />\nmovies:<br />\ngladiator, operation valkyrie, the producers, down periscope.<br />\n<br />\nshows:<br />\nthe borgia, arrested development, game of thrones, monty\npython<br />\n<br />\nmusic:<br />\naesop rock, hail mary mallon, george thorogood and the delaware\ndestroyers, felt<br />\n<br />\nfood:<br />\ni'm down for anything.",food.<br />\nwater.<br />\ncell phone.<br />\nshelter.,duality and humorous things,trying to find someone to hang out with. i am down for anything\nexcept a club.,i am new to california and looking for someone to wisper my secrets\nto.,you want to be swept off your feet!<br />\nyou are tired of the norm.<br />\nyou want to catch a coffee or a bite.<br />\nor if you want to talk philosophy.,"asian, white",75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single,agnosticism
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1. i am a workaholic.<br />\n2. i love to cook regardless of whether i am at work.<br />\n3. i love to drink and eat foods that are probably really bad for\nme.<br />\n4. i love being around people that resemble line 1-3.<br />\ni love the outdoors and i am an avid skier. if its snowing i will\nbe in tahoe at the very least. i am a very confident and friendly.\ni'm not interested in acting or being a typical guy. i have no time\nor patience for rediculous acts of territorial pissing. overall i\nam a very likable easygoing individual. i am very adventurous and\nalways looking forward to doing

As the text data in all essay columns has some noises such as html tags, video links, a function to manage these was created.

In [71]:
cols_to_match = ['age', 'sex', 'orientation', 'diet', 'drinks', 'drugs', 'smokes',  
                 'essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 
                 'essay7', 'essay8', 'essay9', 'religion_cleaned']

def create_match_data(df, cols_to_match):
    match_df = df[cols_to_match]

    essay_columns = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
    # delete all html tags, new line escape, and http links in the essay columns
    filled_df = match_df.replace({r'<[A-Za-z\/][^>]*>' : '', r'\n' : ' ', r'http[^ ]*[ ]' : ' ', r'http[^ ]*' : ''}, regex=True)
    # fill the essay columns with null values with a space
    filled_df.fillna(' ')
    # create the 'combined_essay' column by combining all the essay columns together
    filled_df['combined_essay'] = filled_df[essay_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
              
    return filled_df

In [72]:
match_data = create_match_data(df, cols_to_match)
match_data.columns

Index(['age', 'sex', 'orientation', 'diet', 'drinks', 'drugs', 'smokes',
       'essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6',
       'essay7', 'essay8', 'essay9', 'religion_cleaned', 'combined_essay'],
      dtype='object')

In [73]:
match_data.shape

(59946, 19)

In [93]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import random


def find_top_5_matches(sample, df):
    cols_to_match = ['age', 'sex', 'orientation', 'diet', 'drinks', 'drugs', 'smokes', 'pets',  'essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9', 'religion_cleaned']
    # create a dataframe with only columns used for matching
    data = create_match_data(df, cols_to_match)
    
    # check a user's orientation to prepare proper dataset
    if sample['orientation'] == 'straight':
        dataset = data.loc[(data.sex != sample.sex) & (data.orientation == 'straight')]
    elif sample['orientation'] == 'gay':
        dataset = data.loc[(data.sex == sample.sex) & (data.orientation == 'gay')]
    else:
        dataset = data.loc[((data.sex == sample.sex) & (data.orientation == 'gay')) | 
                        ((data.sex != sample.sex) & (data.orientation == 'straight'))]  
    

    # select only used columns in the dataset by excluding essay0-essay9, and religion columns
    cols_to_transform = ['age', 'orientation','diet', 'drinks', 'drugs', 'smokes', 'pets', 'combined_essay', 'religion_cleaned']
    prepared_dataset = dataset[cols_to_transform]
    
   # remove containing null values rows
    prepared_dataset = prepared_dataset.dropna()
 
    
    # create a columntransformer to transform different features with diferent tools
    ct = ColumnTransformer(transformers=[
        ('tfidf', TfidfVectorizer(), 'combined_essay'),
        ('scaler', StandardScaler(), ['age']), 
        ('onehot', OneHotEncoder(), ['orientation', 'diet', 'drinks', 'drugs', 'smokes', 'pets', 'religion_cleaned'])])

    # fitting columns transformer with new_dataset
    transformed_dataset = ct.fit_transform(prepared_dataset)
    
    # preprocess sample
    essay_columns = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6',
       'essay7', 'essay8', 'essay9']
    for col in essay_columns:
        if not isinstance(sample[col], str):
            sample[col] = ' '
    sample['combined_essay'] = ' '.join([sample[col] for col in essay_columns])

    prepared_sample = sample[cols_to_transform]
      
    # reshape values of sample as it is only one sample
    reshaped_sample = prepared_sample.values.reshape(1,-1)
    df_sample = pd.DataFrame(reshaped_sample, columns = ['age', 'orientation', 'diet', 'drinks', 'drugs', 'smokes', 'pets', 'combined_essay', 'religion_cleaned'])
    transformed_sample = ct.transform(df_sample)
    
    # find distances between sample and all datapoints in prepared 
    distances = []
    for data_point in transformed_dataset:
        distances.append(euclidean_distances(transformed_sample, data_point)[0][0])
    # get top 5 closet indices people in dataset
    five_closet_indice = np.argsort(distances)[::-1][:5]

    print("The top 5 people matches:")
    i = 1
    for index in five_closet_indice:
        print("\n")
        print("Suggested match " + str(i))
        print(dataset.iloc[index])
        i +=1

## Testing a function with sample data

In [84]:
sample = df.iloc[7]
print("Who are you?")
print("\n")
print(sample)

Who are you?


age                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           31
body_type                                                                                                                                                                                                                                               

In [94]:
import numpy as np

find_top_5_matches(sample, df)

<ipython-input-93-22d9eab10116>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['combined_essay'] = ' '.join([sample[col] for col in essay_columns])


The top 5 people matches:


Suggested match 1
age                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            34
sex                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     